In [1]:
import tensorflow as tf 
tf.reset_default_graph()    

In [ ]:
# 模型搭建
batch_size = 1
learning_rate = 0.05
train_w_stddev = 0.3
# display_step = 10 #


n_step = sample_length # h
n_hidden = n_input
n_classes = 1 #跟group一樣多

# placeholder
x = tf.placeholder(tf.float32, [None, n_input, n_step])
y = tf.placeholder(tf.float32, [None, n_classes])

w_in  = tf.Variable(tf.truncated_normal([n_input, n_hidden], mean=0.0, stddev=train_w_stddev, dtype=tf.float32))
w_out = tf.Variable(tf.truncated_normal([n_hidden, n_classes], mean=0.0, stddev=train_w_stddev, dtype=tf.float32))
b_in  = tf.Variable(tf.zeros(n_hidden, dtype = tf.float32))
b_out = tf.Variable(tf.zeros(n_classes, dtype = tf.float32))

X_in = tf.matmul(tf.reshape(x, [-1, n_input]), w_in) + b_in
X_in = tf.reshape(X_in, [-1, n_step, n_hidden])

lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)
init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=init_state, time_major=False)
logits = tf.contrib.layers.fully_connected(  tf.matmul(final_state[1], w_out) + b_out , num_outputs=n_classes, activation_fn=tf.nn.sigmoid)
p_left_and_right = tf.concat(axis=1, values=[logits,1-logits]) 

action = tf.multinomial(logits=tf.log(p_left_and_right), num_samples=1) 
y = 1. - tf.to_float(action) 
optimizer = tf.train.AdamOptimizer(learning_rate) 
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits) 
grads_and_vars = optimizer.compute_gradients(cross_entropy) 
gradients = [variable for grad, variable in grads_and_vars] 
gradient_placeholders = [] 
grads_and_vars_feed = [] 
for grads, variable in grads_and_vars:
#    X = tf.placeholder(tf.float32, shape=[None, n_inputs]) 
    gradient_placeholder = tf.placeholder(tf.float32, shape=variable.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable)) 
training_op = optimizer.apply_gradients(grads_and_vars_feed) 
init = tf.global_variables_initializer() 
saver = tf.train.Saver() 

In [ ]:
# 訓練程式
n_iterations = 1000      # number of training iterations 
     # max steps per episode 
n_games_per_update = 50 # train the policy every 10 episodes 
save_iterations = 10    # save the model every 10 training iterations 
discount_rate = 1
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
samples = []

all_iter_mean_rewards = [0]*int(n_iterations/save_iterations)
with tf.Session() as sess:
#for sss in range(0,1):
    init.run()
    for iteration in range(n_iterations):
        count = [0,0]
        all_actions = []
        all_rewards = []    # all sequences of raw rewards for each episode
        all_gradients = []  # gradients saved at each step of each episode
        for game in range(n_games_per_update):
            current_action = [] 
            current_rewards = []   # all raw rewards from the current episode
            current_gradients = [] # all gradients from the current episode
            x_sample_index = np.random.randint(low=0,high=len(option_C_id_list), size=1)[0]  #取得隨機的sample index
#            this_game_id = option_C_id_list[x_sample_index]
#            print(this_game_id)
#            print(x_sample_index)
#            returns,std_x,n_max_steps = pack_sample(this_game_id,observe,avaulable_asset,variable_names,max_min_compress)
            samples.append(std_x)
            for step in range(0,len(returns[x_sample_index])):
#                action_val, gradients_val = sess.run([action, gradients],feed_dict={X: obs.reshape(1, n_inputs)}) # one obs
                logits_val,p_left_and_right_val,action_val, gradients_val = sess.run([logits,p_left_and_right,action, gradients],feed_dict={x: observes[x_sample_index][step]}) # one obs
#                if action_val[0][0]==1:  
#                    count_1+=1
#                else:
#                    count_0+=1
                
                this_action = action_val[0][0]
                reward = returns[x_sample_index][step][this_action]
#                print(reward)
                current_action.append(this_action)
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
#                if done:
#                    break
            all_actions.append(current_action)
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
        # At this point we have run the policy for 10 episodes, and we are
        # ready for a policy update using the algorithm described earlier.
#        all_rewards = discount_and_normalize_rewards(all_rewards,discount_rate)
        feed_dict = {}
        for var_index, grad_placeholder in enumerate(gradient_placeholders):
            # multiply the gradients by the action scores, and compute the mean
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]for game_index, rewards in enumerate(all_rewards)for step, reward in enumerate(rewards)],axis=0)
            feed_dict[grad_placeholder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict)
        if iteration % save_iterations == 0:
            count_predict = [0,0]
            test1 = 0
            test = 0
            saver.save(sess, "./back_test_option_VOL/my_policy_net_pg1.ckpt")                 
            all_actions_predict = []
            all_rewards_predict = []    # all sequences of raw rewards for each episode
            all_gradients_predict = []  # gradients saved at each step of each episode
            for game in range(0,len(option_C_id_list)):
                current_action_predict = [] 
                current_rewards_predict = []   # all raw rewards from the current episode
                current_gradients_predict = [] # all gradients from the current episode
                x_sample_index = game
                test1+=1
#                returns,std_x,n_max_steps = pack_sample(this_game_id,observe,avaulable_asset,variable_names,max_min_compress)
                samples.append(std_x)
                for step in range(0,len(returns[x_sample_index])):
    #                action_val, gradients_val = sess.run([action, gradients],feed_dict={X: obs.reshape(1, n_inputs)}) # one obs
                    action_val = sess.run(action,feed_dict={x: observes[x_sample_index][step]}) # one obs
                    this_action = action_val[0][0]
                    if this_action==1:  
                        count_predict[1]+=1
                    else:
                        count_predict[0]+=1
                    
                    test +=1
                    reward_predict = returns[x_sample_index][step][this_action] #*(max_min_compress['roi']['max'] -max_min_compress['roi']['min']) + max_min_compress['roi']['min']

                    current_action_predict.append(this_action)
                    current_rewards_predict.append(reward_predict)
                all_actions_predict.append(current_action_predict)
                all_rewards_predict.append(np.mean(current_rewards_predict))
            mean_reward = np.mean(all_rewards_predict)
#            all_iter_mean_rewards[int(iteration/save_iterations)] = np.mean(all_rewards)
            print(test)
            print(str(iteration/n_iterations)+" return:"+str(mean_reward)+" 1 count:"+str(count_predict[1])+" / 0 count:"+str(count_predict[0]))            
